In [ ]:
!pip install datasets
!pip install imblearn

In [ ]:
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd


dataset_name = "OsBaran/tryTr5"  # Dataset adını uygun şekilde değiştir
dataset = load_dataset(dataset_name)


df = pd.DataFrame(dataset['train'])

print("Eksik veri sayısı:\n", df.isnull().sum())

# None/NaN kaldırma
df = df.dropna(subset=['text', 'label'])


print("Temizleme sonrası boyut:", df.shape)


print(df['label'].value_counts())


balanced_train_data = Dataset.from_pandas(df)
dataset_name = "OsBaran/tryTr4"  

dataset = load_dataset(dataset_name)

#DataFrame
df = pd.DataFrame(dataset['train'])

print("Eksik veri sayısı:\n", df.isnull().sum())


df = df.dropna(subset=['text', 'label'])


print("Temizleme sonrası boyut:", df.shape)


print(df['label'].value_counts())


test_data = Dataset.from_pandas(df)
# test_data = load_dataset("OsBaran/tryTr5")["train"]
# train_data, test_data = train_test_split(df, test_size=0, stratify=df['label'], random_state=42)


# print("Eğitim Seti Dağılımı:\n", train_data['label'].value_counts(normalize=True))
# print("Test Seti Dağılımı:\n", test_data['label'].value_counts(normalize=True))


# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=42)
# balanced_train_data, balanced_train_labels = ros.fit_resample(
#     train_data[['text']], train_data['label']
# )


# balanced_train_data = pd.DataFrame(balanced_train_data, columns=["text"])
# balanced_train_data["label"] = balanced_train_labels

# print("Dengeli Eğitim Seti Dağılımı:\n", balanced_train_data['label'].value_counts())
test_data

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "dbmdz/bert-base-turkish-cased"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
from datasets import Dataset


train_dataset = balanced_train_data
test_dataset = test_data

# Kontrol
print(train_dataset)
print(test_dataset)


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = balanced_train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

tokenized_train.set_format("torch")
tokenized_test.set_format("torch")


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    fp16=True  # GPU 

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

trainer.train()
